In [ ]:
# import boto3
# import pandas as pd
# import os
# from botocore.exceptions import NoCredentialsError, ClientError


# credentials = pd.read_csv('IAM Access Keys.csv')

# AWS_S3_BUCKET_NAME='olist--datalake'
# AWS_REGION='ap-south-1'
# AWS_ACCESS_KEY = credentials["Access key ID"].iloc[0]
# AWS_SECRET_KEY = credentials["Secret access key"].iloc[0]
# LOCAL_FILE="dataset/static/olist_customers_dataset.csv"
# NAME_FOR_S3="raw_batch/olist_customers_dataset.csv"


# try:
#     s3_client = boto3.client(
#         service_name="s3",
#         region_name=AWS_REGION,
#         aws_access_key_id=AWS_ACCESS_KEY,
#         aws_secret_access_key=AWS_SECRET_KEY
#     )
# except Exception as e:
#     print("Failed to create S3 client:", e)
#     raise SystemExit


# folder = "dataset/static"

# if not os.path.exists(folder):
#     print(f"Folder '{folder}' does not exist.")
#     raise SystemExit

# for dataset in os.listdir(folder):

#     LOCAL_FILE = f"{folder}/{dataset}"
#     NAME_FOR_S3 = f"raw_batch/{dataset}"

#     if not os.path.isfile(LOCAL_FILE):
#         print(f"⚠ Skipping '{dataset}' (not a file).")
#         continue

#     print(f"⬆Starting upload of '{dataset[:-4]}'...")

#     try:
#         s3_client.upload_file(LOCAL_FILE, AWS_S3_BUCKET_NAME, NAME_FOR_S3)
#         print(f"Upload completed: {dataset}")

#     except FileNotFoundError:
#         print(f"Local file not found: {LOCAL_FILE}")

#     except NoCredentialsError:
#         print("AWS credentials not found or incorrect.")

#     except ClientError as e:
#         print(f"AWS ClientError while uploading '{dataset}': {e}")

#     except Exception as e:
#         print(f"Unexpected error while uploading '{dataset}': {e}")

# print("\n All uploads attempted.")



In [7]:

# import pandas as pd
# from datetime import datetime
# from confluent_kafka import Producer
# import time

# orders = pd.read_csv("dataset/dynamic/olist_orders_dataset.csv")
# items = pd.read_csv("dataset/dynamic/olist_order_items_dataset.csv")
# payments =pd.read_csv("dataset/dynamic/olist_order_payments_dataset.csv")
# reviews = pd.read_csv("dataset/dynamic/olist_order_reviews_dataset.csv")

# conf = {
#     'bootstrap.servers': 'localhost:9092'  
# }

# producer = Producer(conf)


# order_ids = orders["order_id"].tolist()
# batch_size = 2000

# for i in range(0, len(order_ids), batch_size):
    

#     batch_order_ids = order_ids[i:i+batch_size]
    
#     time.sleep(60)
#     # Filter batch
#     batch_orders = orders[orders["order_id"].isin(batch_order_ids)]
#     batch_items = items[items["order_id"].isin(batch_order_ids)]
#     batch_payments = payments[payments["order_id"].isin(batch_order_ids)]
#     batch_reviews = reviews[reviews["order_id"].isin(batch_order_ids)]
    
#     # Convert to CSV string
#     json_orders = batch_orders.to_json(orient="records")
#     json_items = batch_items.to_json(orient="records")
#     json_payments = batch_payments.to_json(orient="records")
#     json_reviews = batch_reviews.to_json(orient="records")
#     print(json_orders)

#     producer.produce('orders',key=f'batch_{i//batch_size +1}',value=json_orders)
#     producer.produce('order_items',key=f'batch_{i//batch_size +1}',value=json_items)
#     producer.produce('order_payments',key=f'batch_{i//batch_size +1}',value=json_payments)
#     producer.produce('order_reviews',key=f'batch_{i//batch_size +1}',value=json_reviews)

#     producer.flush() 
#     print(f"Batch {i//batch_size + 1}: sent {len(batch_order_ids)} orders to Kafka topics")

In [3]:
# from confluent_kafka import Consumer, KafkaException
# import boto3
# import json
# from datetime import datetime
# import pandas as pd


# credentials = pd.read_csv('IAM Access Keys.csv')

# AWS_S3_BUCKET_NAME='olist--datalake'
# AWS_REGION='ap-south-1'
# AWS_ACCESS_KEY = credentials["Access key ID"].iloc[0]
# AWS_SECRET_KEY = credentials["Secret access key"].iloc[0]
# LOCAL_FILE="dataset/static/olist_customers_dataset.csv"
# NAME_FOR_S3="raw_batch/olist_customers_dataset.csv"

# s3_client=boto3.client(service_name='s3',
#                        region_name=AWS_REGION,
#                        aws_access_key_id=AWS_ACCESS_KEY,
#                        aws_secret_access_key=AWS_SECRET_KEY
#                        )


# conf = {
#     'bootstrap.servers': 'localhost:9092',
#     'group.id': 'order-consumer-group',
#     'auto.offset.reset': 'earliest' 
# }

# consumer = Consumer(conf)

# topics = ["orders", "order_items", "order_payments", "order_reviews"]
# consumer.subscribe(topics)

# print("Consumer started. Listening to topics:", topics)

# try:
#     while True:
#         msg = consumer.poll(1.0)  
        
#         if msg is None:
#             continue  
        
#         if msg.error():
#             raise KafkaException(msg.error())

#         print(f"\n📌 Topic: {msg.topic()}")
#         print(f"🔑 Key: {msg.key().decode('utf-8') if msg.key() else None}")
#         print(f"📦 Value: {msg.value().decode('utf-8')[:500]} ...") 

# except KeyboardInterrupt:
#     print("\nClosing consumer...")

# finally:
#     consumer.close()
